In [1]:
# 필요한 라이브러리 불러오기
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score

In [2]:
# 모델 및 데이터 경로 설정
model1_path = "방법 3/model/model1_E_vs_NotE.cbm"
model2_path = "방법 3/model/model2_3class.cbm"
model3_path = "방법 3/model/model3_A_vs_B.cbm"
train_path = "피처_43/피처 선택/43_Segment 추가.csv"
test_path = "피처_43/test_피처43.csv"

In [3]:
# 모델 불러오기
model1 = CatBoostClassifier(); model1.load_model(model1_path)
model2 = CatBoostClassifier(); model2.load_model(model2_path)
model3 = CatBoostClassifier(); model3.load_model(model3_path)

In [4]:
# 데이터 불러오기
df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)

# 피처 컬럼 정의
feature_cols = df_train.columns.drop(['ID', 'Segment'])

# 테스트 데이터에서 피처 추출
X_test = df_test[feature_cols]
df_test_stage1 = df_test.copy()

In [5]:
# 5. model1 예측 (E vs NotE)
pred1 = model1.predict(X_test)
df_test_stage1['Segment_pred'] = pd.Series(pred1.flatten()).map({1: 'E', 0: 'NotE'})

In [6]:
# 6. model2 예측 (C, D, NotCD)
df_not_e = df_test_stage1[df_test_stage1['Segment_pred'] != 'E'].copy()
X_not_e = df_not_e[feature_cols]
pred2 = model2.predict(X_not_e)
df_not_e['Segment_pred'] = pred2.flatten()

In [7]:
# 7. model3 예측 (A vs B)
df_not_cd = df_not_e[~df_not_e['Segment_pred'].isin(['C', 'D'])].copy()
X_not_cd = df_not_cd[feature_cols]
pred3 = model3.predict(X_not_cd)
df_not_cd['Segment_pred'] = pred3.flatten()

In [8]:
# 8. 결과 병합 및 정렬
df_e = df_test_stage1[df_test_stage1['Segment_pred'] == 'E']
df_cd = df_not_e[df_not_e['Segment_pred'].isin(['C', 'D'])]
df_ab = df_not_cd  # A, B 결과

df_final = pd.concat([df_e, df_cd, df_ab]).sort_index()

In [9]:
# 9. 최종 제출 파일 생성
submission = df_final[['ID', 'Segment_pred']].rename(columns={'Segment_pred': 'Segment'})

In [10]:
10/0

ZeroDivisionError: division by zero

In [12]:
# ID별 Segment 다수결
submit_df = (
    submission
    .groupby('ID')['Segment']
    .agg(lambda x: x.value_counts().idxmax())
    .reset_index()
)

In [14]:
submit_df.to_csv("방법 3/submission_method3.csv", index=False, encoding='utf-8-sig')

print("✅ 최종 제출 파일 저장 완료:", submission.shape)

✅ 최종 제출 파일 저장 완료: (600000, 2)
